# A full simulation

In [1]:
import netsquid as ns

print(f'Teleportation example module is located at: {ns.examples.teleportation.__file__}')

Teleportation example module is located at: /home/caitao/anaconda3/lib/python3.8/site-packages/netsquid/examples/teleportation.py


In [5]:
from netsquid.nodes.connections import Connection
from netsquid.components.qchannel import QuantumChannel
from netsquid.qubits import StateSampler
from netsquid.components.qsource import QSource, SourceStatus
from netsquid.components.models import FixedDelayModel, DepolarNoiseModel
import netsquid.qubits.ketstates as ks

## Setting up the complete network

In [6]:
class EntanglingConnection(Connection):
    def __init__(self, length, source_frequency, name='EntanglingConnection'):
        super().__init__(name=name)
        qsource = QSource(f'qsource_{name}', StateSampler([ks.b00], [1.0]), num_ports=2, timing_model=FixedDelayModel(delay=1e9 / source_frequency), status=SourceStatus.INTERNAL)
        self.add_subcomponent(qsource, name='qsource')
        qchannel_c2a = QuantumChannel('qchannel_C2A', length=length/2, models={'delay_model': FibreDelayModel()})
        qchannel_c2b = QuantumChannel('qchannel_C2B', length=length/2, models={'delay_model': FibreDelayModel()})
        # add channels and forward quantum channel output to external port output
        self.add_subcomponent(qchannel_c2a, forward_output=[('A', 'recv')])
        self.add_subcomponent(qchannel_c2b, forward_output=[('B', 'recv')])
        # connect qsource output to quantum channel input
        qsource.ports['qout0'].connect(qchannel_c2a.ports['send'])
        qsource.ports['qout1'].connect(qchannel_c2b.ports['send'])

In [7]:
from netsquid.components import ClassicalChannel

In [8]:
class ClassicalConnection(Connection):
    def __init__(self, length, name='ClassicalConnection'):
        self.add_subcomponent(ClassicalChannel('Channel_A2B', length=length, models={'delay_model': FibreDelayModel()}), forward_input=[('A', 'send')], forward_output=[('B', 'recv')])

In [13]:
import netsquid.components.instructions as instr
from netsquid.components.qprocessor import PhysicalInstruction
from netsquid.components.models.qerrormodels import DepolarNoiseModel
from netsquid.components.models.qerrormodels import DephaseNoiseModel
from netsquid.components.qprocessor import QuantumProcessor

In [14]:
def create_processor(depolar_rate, dephase_rate):
    '''factory to create a quantum processor for each end node
    '''
    measure_noise_model = DephaseNoiseModel(dephase_rate=dephase_rate, time_independent=True)
    physical_instructions = [
        PhysicalInstruction(instr.INSTR_INIT, duration=3, parallel=True),
        PhysicalInstruction(instr.INSTR_H, duration=1, parallel=True, topology=[0, 1]),
        PhysicalInstruction(instr.INSTR_X, duration=1, parallel=True, topology=[0]),
        PhysicalInstruction(instr.INSTR_Z, duration=1, parallel=True, topology=[0]),
        PhysicalInstruction(instr.INSTR_S, duration=1, parallel=True, topology=[0]),
        PhysicalInstruction(instr.INSTR_CNOT, duration=4, parallel=True, topology=[(0, 1)]),
        PhysicalInstruction(instr.INSTR_MEASURE, duration=7, parallel=False, topology=[0], quantum_noise_model=measure_noise_model, apply_q_noise_after=False),
        PhysicalInstruction(instr.INSTR_MEASURE, duration=7, parallel=False, topology=[1])
    ]
    memory_noise_model = DepolarNoiseModel(depolar_rate=depolar_rate)
    processor = QuantumProcessor('quantum_processor', num_positions=2, memory_noise_models=[memory_noise_model] * 2, phys_instructions=physical_instructions)
    return processor

In [17]:
from netsquid.nodes import Node
from netsquid.nodes import Network

In [18]:
def example_network_setup(node_distance=4e-3, depolar_rate=1e7, dephase_rate=0.2):
    '''setup the physical components of the quantum network
    '''
    # setup nodes Alice and Bob with quantum processor
    alice = Node('Alice', qmemory=create_processor(depolar_rate, dephase_rate))
    bob = Node('Bob', qmemory=create_processor(depolar_rate, dephase_rate))
    # create a network
    network = Network('Teleportation_network')
    network.add_nodes([alice, bob])
    network.add_nodes([alice, bob])
    # setup classical connection between nodes
    c_conn = ClassicalConnection(length=node_distance)
    network.add_connection(alice, bob, connection=c_conn, label='classical', port_name_node1='cout_bob', port_name_node2='cin_alice')
    # setup entangling connection between nodes
    source_frequency = 4e4 / node_distance
    q_conn = EntanglingConnection(length=node_distance, source_frequency=source_frequency)
    port_ac, port_bc = network.add_connection(alice, bob, connection=q_conn, label='quantum', port_name_node1='qin_charlie', port_name_node2='qin_charlie')
    alice.ports[port_ac].forward_input(alice.qmemory.ports['qin1'])
    bob.ports[port_bc].forward_input(bob.qmemory.ports['qin0'])
    return network

## Setting up the simulation